In [24]:
import torch
import torch.nn.functional as F
import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else "cpu"
print(device)

block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-4
eval_iters = 250

cuda


In [25]:
with open("sorcerers_stone.txt", "r", encoding="utf-8") as f:
    text = f.read()

chars = sorted(set(text))
vocab_size = len(chars)
print(chars)

['\t', '\n', ' ', '!', '"', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '\\', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~']


In [26]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [27]:
print(encode("hello world"))

[59, 56, 63, 63, 66, 2, 74, 66, 69, 63, 55]


In [28]:
data = torch.tensor(encode(text), dtype=torch.long)

In [29]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [30]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [31]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)


        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index


In [32]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

	6)w?PUCSy?nbRMZyrCcLR(79X0bk!*2	DH6wnmZ~)bBFhxXdoc?o*X3F0nJ?fc(.3HkApMtPI	TR:gCVNMb
X9\DTLhj6BL"tLlnk.LNO0ISsNUe-J0O,XF2 FN-eWEY(2Os\QzHSf:n"-\Q~wg6H~SYZ"b'9\a1W6Y1,tZw.0Ck7:zE,mxM
YE;cOA,r 8 xJTFN;v(kN-!c04UmPIa)Rq	y!TELmx(aFQayN-gctA-A9iqP0hseoH8(P2USK0KXFNnp-8K8ufV'~D.EY1WQr4ZH6FU2ps3a9o?F
d8YVj-as7:EY23x?L V1QoK,v65TMiGghsp(!JuqfXFC0ktN.E	FBfaCLC.QafLG.K;?jDalX,
UniodTgz,'f?Ud6L(kN-1"wajpa	h
8Y:FgP(F31Iqk)P0*~~(gnmP3aTFT4c'	O91zUF415O,O*!cN-0kBmCVD,"YVSx3(Da~XguxXFC"2!5IyRW*FNFV(qP.3pvV'SHf~


In [34]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # getting a sample batch of data
    xb, yb = get_batch('train')

    # evaluating the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.642, val loss: 4.634
step: 250, train loss: 4.571, val loss: 4.555
step: 500, train loss: 4.509, val loss: 4.499
step: 750, train loss: 4.469, val loss: 4.451
4.384072303771973
